# Week 7 Coding Exercises

This week we learned three main skills:

- Regular Expressions
- Pulling Webpages
- Scraping Webpages

I want you to go further with the code we started developing in the coding tutorial.

By the end of the tutorial, we had code to get all of the books, along with their price, rating, and a link to their dedicated page.

My next challenge for you is to do the following

1. On each of the dedicated pages, there is a description of the book. Write code that will scrape the description for each book and add it as a column on the product_info dataframe.
2. Currently, this scraper can only access the first page of results. Modify the scraper so that it iterates through pages. (Hint: There is a link close to the bottom of each page, but the title will be different each time (page-3, page-4, etc.). You will need to work in a solution that finds the link each time, or automatically increments the page at each iteration.

In [ ]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

url = "http://books.toscrape.com"
session = requests.Session()
page = session.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
product_pods = soup.find_all('article', class_="product_pod")

def get_product_info(product_pod):
    # Title can be accessed from img alt
    image_elem = product_pod.div.a.img
    title = image_elem['alt']
    # Rating can be accessed from class (css) on star-rating
    rating_elem = product_pod.find('p', class_=re.compile(r"star-rating .*"))
    rating = rating_elem['class'][1]+"/Five" # Second class attribute
    price_elem = product_pod.find('div', class_="product_price")
    price = re.search(re.compile("[0-9\.]+"), price_elem.text)[0]
    link = product_pod.find('a', href=True)['href']
    return title, rating, price, link

product_info = []
for pod in product_pods:
    product_info.append(get_product_info(pod))

product_info = pd.DataFrame(product_info, columns=["Title", "Rating", "Price", 'Link'])
product_info.loc[:, 'Link'] = product_info['Link'].apply(lambda x: url+"/"+x)

Regex Bonus Challenge:

Full disclaimer: I wasn't able to figure out a solution to this, but I thought you might enjoy the challenge.

Given the lyrics in the box below, find a way to match the full phrase after each "million".

Therefore you should get back the matches:

```
['bags of the best Sligo rags',
 'barrels of stone',
 'sides of old blind horses hides',
 'barrels of bones',
 'hogs',
 'dogs',
 'barrels of porter',
 'bails of old nanny goats']
 ```


The closest I was able to come was: `million ([\w ]+)`, but this matches the `and seven million dogs`, which I want separately.

In [ ]:
lyrics = """
On the Fourth of July, 1806
We set sail from the sweet Cove of Cork
We were sailing away with a cargo of bricks
For the Grand City Hall in New York
'Twas a wonderful craft, she was rigged fore and aft
And oh, how the wild wind drove her
She stood several blasts, she had twenty seven masts
And they called her The Irish Rover
We had one million bags of the best Sligo rags
We had two million barrels of stone
We had three million sides of old blind horses hides
We had four million barrels of bones
We had five million hogs and six million dogs
Seven million barrels of porter
We had eight million bails of old nanny goats' tails
In the hold of the Irish Rover
"""

r = re.compile(r"million ([\w ]+)") # This solution doesn't work

re.findall(r, lyrics)